In [1]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon

In [19]:
import ais_filtering

## FUNCTIONS


In [2]:
def mini_cleaning_pipeline(df: pd.DataFrame) -> pd.DataFrame:

    df = df[df["Type of mobile"].isin(["Class A", "Class B"])].drop(columns=["Type of mobile"])

    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

    df = df.drop_duplicates(["Timestamp", "MMSI", ], keep="first")
    df = df.drop(columns=df.columns[-5:])

    knots_to_ms = 0.514444
    df["SOG"] = knots_to_ms * df["SOG"]
    
    return df
    
def quick_summary(df: pd.DataFrame):
    """
    Generate a quick summary of the AIS data in the DataFrame.
    Args:
        df: DataFrame with AIS data, must contain 'MMSI', 'Timestamp', 'Latitude', 'Longitude' columns.
        
    Returns:
        None: Prints summary statistics.
        """
    # Number of unique vessels
    num_vessels = int(df['MMSI'].nunique())
    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

    # Spatial extent
    lat_min, lat_max = df['Latitude'].min(), df['Latitude'].max()
    lon_min, lon_max = df['Longitude'].min(), df['Longitude'].max()
    centroid_lat, centroid_lon = df['Latitude'].mean(), df['Longitude'].mean()

    # Messages per vessel
    msgs_per_vessel = df.groupby('MMSI').size()
    msgs_stats = msgs_per_vessel.describe().to_dict()
    top_10_vessels = msgs_per_vessel.sort_values(ascending=False).head(10)

    # Print concise overview
    print(f"Number of unique vessels: {num_vessels}")
    print(f"Latitude range: {lat_min:.6f} -- {lat_max:.6f}")
    print(f"Longitude range: {lon_min:.6f} -- {lon_max:.6f}")
    print(f"Centroid (lat, lon): ({centroid_lat:.6f}, {centroid_lon:.6f})")
    print("\nMessages per vessel (summary):")
    for k, v in msgs_stats.items():
        print(f"  {k}: {v}")
    print("\nTop 10 vessels by number of messages (MMSI: count):")
    print(top_10_vessels.to_string())


def singularize_vessels(df, mmsi_col="MMSI", start_col_index=11, join_conflicts=True, sep=" | "):
    cols = list(df.columns[start_col_index:])
    if mmsi_col not in df.columns:
        raise KeyError(f"MMSI column '{mmsi_col}' not found in dataframe")

    def _agg(series):
        vals = series.dropna().unique().tolist()
        if len(vals) == 0:
            return np.nan
        if len(vals) == 1:
            return vals[0]
        if join_conflicts:
            return sep.join(map(str, vals))
        return vals

    grouped = df.groupby(mmsi_col)[cols].agg(_agg).reset_index()
    ordered_cols = [mmsi_col] + cols
    return grouped[ordered_cols]

def filter_inside_square(df, bbox) -> pd.DataFrame:
    north, west, south, east = bbox
    df = df[(df["Latitude"] <= north) & (df["Latitude"] >= south) & (df["Longitude"] >= west) & (df["Longitude"] <= east)] 
    return df

def is_inside_polygon(lat, lon, polygon_coords):
    """
    Check if a point (lat, lon) is inside a polygon.
    
    Args:
        lat: Latitude
        lon: Longitude
        polygon_coords: List of (lon, lat) tuples defining polygon vertices
        
    Returns:
        Boolean: True if point is inside polygon
    """
    point = Point(lat, lon)
    polygon = Polygon(polygon_coords)
    return polygon.contains(point)

# 🛰️ AIS Data Filtering and Preprocessing — Kattegat Submarine Cable Area

This notebook documents the initial steps of data preparation for the AIS anomaly detection project focusing on the submarine communication cables in the Kattegat area (Denmark–Sweden).

The goal is to isolate AIS data within a defined polygon surrounding three specific cable routes (**GC2**, **Kattegat 2A**, and **Kattegat 2B**) and perform preliminary cleaning and filtering.


## 🧭 Area of Interest Definition

We are interested in identifying potential anomalous or risky vessel behaviors near submarine cables.

By Danish law, a 200 m protection zone exists on each side of these cables, but in this study we extend the inspection zone to **5 km** to capture behavioral precursors such as early anchoring, trawling, or route deviations before ships enter the restricted corridor.

The polygon defining our area of interest was manually approximated based on the **DKCPC map**.

It includes a section of the Danish coast near Saeby and extends to the Swedish side around Lerkil, covering the main cable routes.  
The coordinates are stored as `(latitude, longitude)` pairs and define an octagonal region enclosing the study zone.


In [4]:
#initial filter on bounding box (take northest and southest, westest and eastest points):
bbox = [57.58, 10.5, 57.12, 11.92]  # north, west, south, east
# Define polygon coordinates as (lat, lon)
polygon_coords = [
    (57.3500, 10.5162),  # coast top left
    (57.5120, 10.9314),  # sea top left
    (57.5785, 11.5128),  # sea top right
    (57.5230, 11.9132),  # top right (Swedish coast)
    (57.4078, 11.9189),  # bottom right (Swedish coast)
    (57.1389, 11.2133),  # sea bottom right
    (57.1352, 11.0067),  # sea bottom left
    (57.1880, 10.5400),  # coast bottom left
    (57.3500, 10.5162),  # close polygon (duplicate of first)
]

## ⚙️ Data Loading

In this step, AIS data (in CSV format) is loaded and receives a first hand cleaning 10% of the data

In [5]:
df_28 = pd.read_csv("ais-data/aisdk-2025-10-28.csv")

In [6]:
print(f"No filtering: {len(df_28):,} rows, {df_28['MMSI'].nunique():,} unique vessels, {df_28.shape[1]} columns")

No filtering: 15,646,647 rows, 2,961 unique vessels, 26 columns


In [7]:
df_cleaned = mini_cleaning_pipeline(df_28)

In [8]:
print(f" Initial filtering complete: {len(df_cleaned):,} rows, {df_cleaned['MMSI'].nunique():,} unique vessels, {df_cleaned.shape[1]} columns")

 Initial filtering complete: 8,978,067 rows, 2,777 unique vessels, 20 columns


## 📍 SQUARE Filtering

The box coordinates defined earlier are used to filter the AIS dataset.
Each point is checked to determine whether it falls inside the box using simple operations

In [9]:
df_inside_square = filter_inside_square(df_cleaned, bbox)

In [10]:
print(f" Bounding box filtering complete: {len(df_inside_square):,} rows, {df_inside_square['MMSI'].nunique():,} unique vessels, {df_inside_square.shape[1]} columns")

 Bounding box filtering complete: 562,116 rows, 253 unique vessels, 20 columns


## 📍 Polygon Filtering

The polygon coordinates defined earlier are used to filter the AIS dataset.

Each point is checked to determine whether it falls inside the polygon using geometric operations (e.g. with the `shapely` library).

The resulting dataset contains only positions **within the defined Kattegat zone**, roughly **3–4 %** of the original AIS dataset.


In [11]:
df_inside_poly = df_inside_square[
    df_inside_square.apply(lambda row: is_inside_polygon(row['Latitude'], row['Longitude'], polygon_coords), axis=1)
]

In [12]:
print(f" Polygon filtering complete: {len(df_inside_poly):,} rows, {df_inside_poly['MMSI'].nunique():,} unique vessels, {df_inside_poly.shape[1]} columns")

 Polygon filtering complete: 314,285 rows, 172 unique vessels, 20 columns


## 📊 Data Overview After Filtering

We summarize the filtered dataset:

- Number of unique vessels  
- Time coverage of the subset  
- Spatial extent and approximate number of messages per vessel  

This quick overview helps confirm that the filtering worked as expected and that the dataset is representative of the study area.


In [13]:
quick_summary(df_inside_poly)

Number of unique vessels: 172
Latitude range: 57.163097 -- 57.578258
Longitude range: 10.517063 -- 11.867375
Centroid (lat, lon): (57.361891, 11.230398)

Messages per vessel (summary):
  count: 172.0
  mean: 1827.2383720930231
  std: 2845.5646981958407
  min: 2.0
  25%: 544.75
  50%: 633.0
  75%: 1068.25
  max: 17475.0

Top 10 vessels by number of messages (MMSI: count):
MMSI
219009229    17475
220279000    11712
220323000    10512
220046000     9308
219010207     9083
219006219     8854
220334000     8836
219001553     8814
219000543     8698
220349000     8662


# USING SUMMARY FUNCTIONS TO FILTER AND DIVIDE

In [6]:
import pandas as pd
import ais_filtering

In [7]:
df_filtered = ais_filtering.df_filter(df_28, True, False)

Before filtering: 15,646,647 rows, 2,961 unique vessels


MemoryError: Unable to allocate 119. MiB for an array with shape (1, 15646647) and data type object

In [1]:
df_static, df_dynamic = ais_filtering.split_static_dynamic(df_filtered, join_conflicts=True, sep=" | ")

NameError: name 'ais_filtering' is not defined